<a href="https://colab.research.google.com/github/NattyPavez/Agente-Gemini-CV-conversor-portafolio/blob/main/Proyecto_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto paralelo a curso Inmersión AI de Alura con Google Gemini

Primera parte

Librerías utilizadas

In [1]:
!pip install -q langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install email-validator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 8.9 MB/s eta 0:00:00


In [3]:
#permite leer PDF
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 42.8 MB/s eta 0:00:00


In [4]:
# permite crear una interfaz simple con muy pocas líneas de código
!pip install -q gradio

Importación api key de AIStudio

In [5]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('personal_gemini_api_key')

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [7]:
# Importa la clase de incrustaciones
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

#  modelo de incrustaciones para la clasificación de perfiles
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
    )

### Asistente de recuperacion de datos de Curriculum Vitae para generar portafolio profesional

In [8]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional

class Habilidades(BaseModel):
    tecnicas: List[str] = Field(default_factory=list)
    blandas: List[str] = Field(default_factory=list)

class ExperienciaLaboral(BaseModel):
    empresa_o_proyecto: str
    cargo: str
    periodo: str
    descripcion: str

class Educacion(BaseModel):
    institucion: str
    titulo: str
    periodo: str

class Perfil(BaseModel):
    nombre: str
    email: EmailStr
    telefono: Optional[str] = None
    ciudad_pais: str

class CurriculumVitae(BaseModel):
    perfil: Perfil
    resumen: str
    experiencia_laboral: List[ExperienciaLaboral] = Field(default_factory=list)
    educacion: List[Educacion] = Field(default_factory=list)
    habilidades: Habilidades
    puntos_clave_sugeridos: List[str] = Field(default_factory=list)



In [9]:
PROMPT_BASE_EXTRACCION = """Eres un asistente experto en análisis de currículums. Tu tarea es extraer la información clave de un currículum vitae y estructurarla en un objeto JSON.

La información que debes extraer es:
1. **Información personal**: Nombre completo, email, teléfono, ciudad y país.
2. **Resumen profesional**: Un breve resumen que destaque las principales habilidades y experiencia.
3. **Experiencia laboral**: Para cada puesto, extrae el nombre de la empresa, cargo, período (ej: "2018 - 2022") y una descripción breve de las responsabilidades y logros clave.
4. **Educación**: Para cada título, extrae el nombre de la institución, el título obtenido y el período (ej: "2015 - 2018").
5. **Habilidades**: Lista de habilidades técnicas (ej: "Python", "SQL", "Git") y habilidades blandas (ej: "Liderazgo", "Comunicación").

Después de extraer la información, evalúa la profesión del currículum y sugiere qué información adicional sería valiosa para destacar en un portafolio web. Estas sugerencias deben basarse en estándares de la industria para esa profesión. Devuelve esta sugerencia en un campo llamado "puntos_clave_sugeridos".

Si el perfil detectado es '{perfil_detectado}', asegúrate de adaptar la extracción de la siguiente manera:
{instruccion_especifica}
"""

Clasificación de perfiles utilizando Emmbeddins

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import Dict
from langchain_core.messages import SystemMessage, HumanMessage

# Perfiles de referencia
perfiles_referencia = {
    "Perfil Administrativo / Asistente": "Profesional con experiencia en gestión de agendas, coordinación de equipos y atención al cliente. Destacada capacidad organizativa, manejo de herramientas ofimáticas (Excel, Word, Power Point) y optimización de procesos administrativos.",
    "Perfil Comercial / Ventas": "Ejecutivo de ventas con trayectoria en el cumplimiento de metas comerciales, negociación, cierre de tratos y fidelización de clientes. Orientado a resultados y con fuerte habilidad para generar nuevas oportunidades de negocio, prospección y gestión de cartera.",
    "Perfil Tecnológico / TI": "Desarrollador de software con experiencia en lenguajes de programación como Java, Python, JavaScript. Manejo de frameworks como Spring Boot, Django, o React. Experto en bases de datos SQL y NoSQL, desarrollo de APIs y metodologías ágiles (Scrum, Kanban).",
    "Perfil Marketing / Comunicación": "Especialista en marketing digital con dominio de campañas en redes sociales (Facebook Ads, Google Ads), SEO, SEM y analítica web. Capacidad para crear estrategias de contenido que aumentan la visibilidad y el engagement de marcas, email marketing y relaciones públicas.",
    "Perfil Financiero / Contabilidad": "Contador auditor con sólida experiencia en análisis financiero, control de gastos, auditoría, planificación tributaria y cumplimiento normativo. Fuerte orientación a la precisión, la eficiencia en procesos contables y la elaboración de reportes financieros.",
    "Perfil de Salud": "Profesional de la salud con experiencia en atención clínica, diagnóstico, tratamiento y enfoque en la calidad del servicio al paciente. Capacitado en protocolos de emergencia, prevención de enfermedades, promoción de la salud y educación a la comunidad.",
    "Perfil de Recursos Humanos": "Especialista en gestión de talento, con experiencia en reclutamiento y selección, capacitación, evaluación de desempeño y clima organizacional. Enfocado en potenciar equipos de alto desempeño y en la aplicación de políticas de bienestar laboral.",
    "Perfil de Educación / Docencia": "Docente con experiencia en la enseñanza de asignaturas como [ej: matemáticas, historia] en [ej: nivel escolar o universitario]. Capaz de diseñar planes pedagógicos efectivos, aplicar metodologías innovadoras y adaptarse a distintos estilos de aprendizaje. Enfocado en la motivación y el desarrollo integral de los estudiantes.",
    "Perfil de Logística / Operaciones": "Profesional en logística con trayectoria en gestión de inventarios, control de bodegas, cadena de suministro y optimización de procesos de distribución. Orientado a la eficiencia operativa, reducción de costos, cumplimiento de tiempos de entrega y manejo de sistemas ERP.",
    "Perfil de Atención al Cliente / Servicios": "Especialista en atención al cliente con experiencia en resolución de problemas, manejo de reclamos y generación de experiencias positivas. Destacado por la empatía, comunicación efectiva, orientación a la satisfacción del usuario y manejo de sistemas CRM."
}

# incrustaciones una sola vez al inicio del programa
vectores_referencia = {
    perfil: embeddings_model.embed_query(texto)
    for perfil, texto in perfiles_referencia.items()
}

def clasificar_perfil(texto_cv: str) -> str:
    """Clasifica el perfil de un CV comparando su incrustación con perfiles de referencia."""
    if not texto_cv:
        return "Perfil no definido"
    try:
        vector_cv = embeddings_model.embed_query(texto_cv)
        mejor_coincidencia = "No se pudo clasificar"
        mayor_similitud = -1
        for perfil, vector_ref in vectores_referencia.items():
            similitud = cosine_similarity(np.array(vector_cv).reshape(1, -1), np.array(vector_ref).reshape(1, -1))[0][0]
            if similitud > mayor_similitud:
                mayor_similitud = similitud
                mejor_coincidencia = perfil
        print(f"El sistema ha clasificado este perfil como: {mejor_coincidencia} (Similitud: {mayor_similitud:.2f})")
        return mejor_coincidencia
    except Exception as e:
        print(f"Ocurrió un error en la clasificación: {e}")
        return "Error en la clasificación"

llm_cv_chain = llm.with_structured_output(CurriculumVitae)

def extraer_y_clasificar_cv(texto_cv: str):
    """
    Función principal que clasifica el CV y extrae los datos con un prompt dinámico.
    """
    perfil_clasificado = clasificar_perfil(texto_cv)
    print(f"Perfil detectado: {perfil_clasificado}")

    if "Tecnológico" in perfil_clasificado or "freelance" in texto_cv.lower():
        instruccion = "Para la experiencia laboral, si se menciona un proyecto o cliente, usa el nombre del proyecto en lugar del nombre de la empresa. En las habilidades, prioriza los lenguajes de programación, frameworks y herramientas tecnológicas."
    else:
        instruccion = "Sigue el formato estándar de experiencia laboral, usando el nombre de la empresa en cada puesto."

    prompt_dinamico = PROMPT_BASE_EXTRACCION.format(perfil_detectado=perfil_clasificado, instruccion_especifica=instruccion)

    salida = llm_cv_chain.invoke([
        SystemMessage(content=prompt_dinamico),
        HumanMessage(content=texto_cv)
    ])

    return salida

Prueba con CV cargado en los archivos de *Colab*

In [11]:
import fitz

def leer_pdf(ruta_del_archivo):
    """
    Lee un archivo PDF y extrae todo su texto en una sola cadena.
    """
    try:
        documento = fitz.open(ruta_del_archivo)
        texto_completo = ""
        for pagina in documento:
            texto_completo += pagina.get_text()
        return texto_completo
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta '{ruta_del_archivo}'.")
        return ""

ruta_del_pdf = "CV_Sofia_Allende_Ramirez.pdf" #  nombre de tu archivo

# Extrae el texto del PDF
texto_del_cv = leer_pdf(ruta_del_pdf)

if texto_del_cv:
    try:
        # Usa tu función para procesar el texto
        datos_extraidos = extraer_y_clasificar_cv(texto_del_cv)

        # Imprime el resultado para verificar que funciona
        print("\nExtracción de datos exitosa:")
        print(datos_extraidos.model_dump_json(indent=2))
    except Exception as e:
        print(f"Ocurrió un error inesperado al procesar el CV: {e}")

El sistema ha clasificado este perfil como: Perfil de Salud (Similitud: 0.72)
Perfil detectado: Perfil de Salud

Extracción de datos exitosa:
{
  "perfil": {
    "nombre": "Sofía Allende Ramírez",
    "email": "sofia.allende.vet@email.com",
    "telefono": "+56 9 8765 4321",
    "ciudad_pais": "Valdivia, Chile"
  },
  "resumen": "Veterinaria con 12 años de experiencia dedicada al cuidado de animales de compañía y grandes. Especializada en medicina interna y procedimientos de emergencia, con un fuerte enfoque en el bienestar animal y la educación de los dueños. Busco unirme a un equipo comprometido en la región de Valdivia, donde pueda aportar mi experiencia en clínicas y mi pasión por el trato directo con los animales.",
  "experiencia_laboral": [
    {
      "empresa_o_proyecto": "Clínica Veterinaria El Roble",
      "cargo": "Jefa de Clínica",
      "periodo": "2018 - Presente",
      "descripcion": "Lideré un equipo de 5 veterinarios y 3 técnicos, supervisando las operaciones diaria

Permitir carga de CV por interfaz

In [12]:
import gradio as gr
from typing import IO
import fitz
import json

def leer_pdf(ruta_del_archivo):
    """
    Lee un archivo PDF y extrae todo su texto en una sola cadena.
    """
    try:
        documento = fitz.open(ruta_del_archivo)
        texto_completo = ""
        for pagina in documento:
            texto_completo += pagina.get_text()
        return texto_completo
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        return ""

# Modificamos la función para que retorne la clasificación y el JSON
def extraer_y_clasificar_cv(texto_cv: str):
    """
    Función principal que clasifica el CV y extrae los datos con un prompt dinámico.
    """
    # El modelo de incrustaciones clasifica el perfil
    perfil_clasificado = clasificar_perfil(texto_cv)
    print(f"Perfil detectado: {perfil_clasificado}")

    if "Tecnológico" in perfil_clasificado or "freelance" in texto_cv.lower():
        instruccion = "Para la experiencia laboral, si se menciona un proyecto o cliente, usa el nombre del proyecto en lugar del nombre de la empresa. En las habilidades, prioriza los lenguajes de programación, frameworks y herramientas tecnológicas."
    else:
        instruccion = "Sigue el formato estándar de experiencia laboral, usando el nombre de la empresa en cada puesto."

    prompt_dinamico = PROMPT_BASE_EXTRACCION.format(perfil_detectado=perfil_clasificado, instruccion_especifica=instruccion)

    salida = llm_cv_chain.invoke([
        SystemMessage(content=prompt_dinamico),
        HumanMessage(content=texto_cv)
    ])

    return perfil_clasificado, salida

def procesar_cv_desde_interfaz(archivo_pdf: IO):
    """
    Función que recibe un archivo PDF desde la interfaz, lo lee y lo procesa.
    """
    if archivo_pdf is None:
        return "Por favor, sube un archivo PDF.", ""  # Retorna dos valores
    try:
        texto_del_cv = leer_pdf(archivo_pdf.name)

        if not texto_del_cv:
            return "El archivo PDF no contiene texto extraíble.", "" # Retorna dos valores

        # Llamas a la función principal que ahora devuelve dos valores
        perfil_detectado, datos_extraidos = extraer_y_clasificar_cv(texto_del_cv)

        # Retornamos los dos valores esperados por Gradio
        return perfil_detectado, datos_extraidos.model_dump_json(indent=2)
    except Exception as e:
        return f"Ocurrió un error al procesar el archivo: {e}", ""

# Creamos la interfaz de Gradio con múltiples salidas
interfaz = gr.Interface(
    fn=procesar_cv_desde_interfaz,
    inputs=gr.File(type="filepath", label="Sube tu CV en formato PDF"),
    outputs=[
        gr.Textbox(label="Clasificación del perfil"),
        gr.Json(label="Datos del CV extraídos")
    ],
    title="Asistente para Creación de Portafolios",
    description="Sube tu currículum vitae en PDF y obtén los datos estructurados en formato JSON."
)

# Lanzamos la interfaz.
interfaz.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9346a074c4388cb7ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Segunda parte